In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hit_rates as hr
import rankfmlib as fmlib
# new library for performance studies built from rankfmlib to read a single file rather than yearly files.
import newlib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from rankfm.rankfm import RankFM
import function_lib as flib

from rankfm.evaluation import hit_rate

Read member attributes

In [3]:
%%time
# Perhaps add an argument stating which colums are attributes
interaction_dct = newlib.read_data_attributes_single_file("activity_reduced_with_attributes.csv",
                                                        age_cuts=[0,30,50,70,150])

CPU times: user 940 ms, sys: 155 ms, total: 1.09 s
Wall time: 1.09 s


The problem to solve: Why are results worse with filter=False? To figure this out, I may have to write my own recommender and hit_rate functions. 
 1) optimize parameter without attributes
 2) optimize age brackets using just one attribute
 3) optimize temperature brackets (destination features) using just one attribute
 4) add gender, country of origin to the member attributes
 
 One problem found was that I was removing duplicate Member/Destinations too early. They must be removed from 
 the training and validation sets separately. Now I get up to 40% if data is not filtered. However, I only get 10% accuracy
 when the previous data is filtered. WHY? 

___

# Create newlib.recommender(data_train)

In [4]:
interaction_dct.keys()

dict_keys(['df_members', 'df_user_attr', 'df_item_attr'])

In [15]:
%%time
# Add a percentage offset that will not be used. 
newlib.train_valid_dct(interaction_dct, train_perc=0.3, valid_perc=0.3, temporal=False)
print(interaction_dct.keys())

(714086, 2)
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test', 'train_dest_sets', 'valid_dest_sets'])
CPU times: user 82.2 ms, sys: 104 µs, total: 82.3 ms
Wall time: 81.7 ms


In [16]:
loss = 'warp'
loss = 'bpr'  # nb neg samples = 1
model = RankFM(factors=20, loss=loss, max_samples=10000, alpha=0.05, beta=0.1, learning_rate=0.1, learning_schedule='constant')

1. create dataframe list list of destinations for each member. Do this for training and validation

In [19]:
%%time
newlib.run_model(model, interaction_dct, nb_epochs=100, topN=5, with_attrib=True)
#topN=5, nb_epochs=500, hr=0.16 (filtered), hr=0.64 (not filtered), with_attrib=True`
# better results without attributes (age). Perhaps must improve on subdivision. Start experiments. 
# Add gender and country of origin. 
# Experiment with training/validation split. Two different types: 
#  1) Keeping training fixed, vary validation (should not change much)
#  2) Keep training and validation the same, vary initial offset. (set test_error to zero)
# Test different age splits
# Put Read data, train_valid, run_model into a single method. 
# Create dictionary for each run with all the parameters (train_perc, valid_perc, temporal, use_attrib, learning_schedule, 
# learning_rate, alpha, beta, loss, factors, run_nb if repeated for mean/variance)
# No temporal (lose 3% on both filtered and non-filtered)
# I might have to run multiple times to estimate variance. Say, run 10 times.

# Compare against the neural net version of fm, which can run adamW. See if I get the same results. 
# Interface this program with the otherone that uses a DataLoader

hr (previous filtered):  0.35792852023805816
hr (previous not filtered):  0.6907736889664413
CPU times: user 7.73 s, sys: 3.38 ms, total: 7.73 s
Wall time: 7.73 s


In [12]:
%%time
# no filter: 63% accurate. That makes sense. 
# with filter of hits in training set: 11% (same results as those of rankfm)
# Same answers when repeated multiple times
result = newlib.recommender(model, interaction_dct, keep_nb_members=None, topN=7)
# topN=5, nb_epochs=500, hr=0.11 (filtered), hr=0.6 (not filtered), with_attrib=True

hit rate (without previous filter) =  0.6998566955044849
hit rate (with previous filter) =  0.39822816288467516
CPU times: user 16.9 s, sys: 225 ms, total: 17.1 s
Wall time: 17 s


In [ ]:
%%time
# no filter: 63% accurate. That makes sense. 
# with filter of hits in training set: 11% (same results as those of rankfm)
result = newlib.recommender(model, interaction_dct, keep_nb_members=None, topN=5)